In [37]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

## Cargamos Dataset y visualizamos las primeras 5 filas

In [38]:
db = pd.read_csv("train.csv")
print("Dimensionalidad:",db.shape)
db.head()

Dimensionalidad: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Tratamiento previo del Dataset


In [39]:
# Eliminar columna PassengerId
db.drop("PassengerId",axis=1,inplace=True)

In [40]:
# calculamos % de duplicados en name
print("Duplicados en 'Name':",db["Name"].duplicated().sum()/db.shape[0]*100,"%")

Duplicados en 'Name': 0.0 %


In [41]:
# Poner columna name como indice
db.set_index("Name",inplace=True)

In [42]:
# Mirar % de datos nulos por columna
for col in db.columns:
    print(col,":",db[col].isna().sum()/db.shape[0]*100,"%")

Survived : 0.0 %
Pclass : 0.0 %
Sex : 0.0 %
Age : 19.865319865319865 %
SibSp : 0.0 %
Parch : 0.0 %
Ticket : 0.0 %
Fare : 0.0 %
Cabin : 77.10437710437711 %
Embarked : 0.22446689113355783 %


In [43]:
# Cabina no nula y Pclass = 1
print("No nulos con 'Pclass' = 1:",db[(db["Cabin"].notnull()) & (db["Pclass"]==1)].shape[0]/db[db["Cabin"].notnull()].shape[0] * 100,"%")

# Eliminamos la columna Cabin
db.drop("Cabin",axis=1,inplace=True)

No nulos con 'Pclass' = 1: 86.27450980392157 %


In [44]:
# tabla donde la edad es nula
db[db["Age"].isna()]


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
Name,,,,,,,,,
"Moran, Mr. James",0,3,male,NaN,0,0,330877,8.4583,Q
"Williams, Mr. Charles Eugene",1,2,male,NaN,0,0,244373,13.0000,S
"Masselmani, Mrs. Fatima",1,3,female,NaN,0,0,2649,7.2250,C
"Emir, Mr. Farred Chehab",0,3,male,NaN,0,0,2631,7.2250,C
"O'Dwyer, Miss. Ellen ""Nellie""",1,3,female,NaN,0,0,330959,7.8792,Q
...,...,...,...,...,...,...,...,...,...
"Razi, Mr. Raihed",0,3,male,NaN,0,0,2629,7.2292,C
"Sage, Miss. Dorothy Edith ""Dolly""",0,3,female,NaN,8,2,CA. 2343,69.5500,S
"van Melkebeke, Mr. Philemon",0,3,male,NaN,0,0,345777,9.5000,S


In [45]:
# Imprimimos las filas de Embarked que tienen valores nulos
db[db["Embarked"].isna()]

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
Name,,,,,,,,,
"Icard, Miss. Amelie",1,1,female,38.0,0,0,113572,80.0,NaN
"Stone, Mrs. George Nelson (Martha Evelyn)",1,1,female,62.0,0,0,113572,80.0,NaN


https://www.encyclopedia-titanica.org/titanic-survivor/amelia-icard.html

https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html

Criada de, por ello el mismo ticket. Al ser supervivientes hay fácil acceso a información.

In [46]:
#Rellanamos los valores nulos de Embarked con S de Southampton
db["Embarked"].fillna("S",inplace=True)

In [47]:
# Mirar % de datos nulos por columna
for col in db.columns:
    print(col,":",db[col].isnull().sum()/db.shape[0]*100,"%")

Survived : 0.0 %
Pclass : 0.0 %
Sex : 0.0 %
Age : 19.865319865319865 %
SibSp : 0.0 %
Parch : 0.0 %
Ticket : 0.0 %
Fare : 0.0 %
Embarked : 0.0 %


In [48]:
# Calculamos % de duplicados en ticket
print("Duplicados en 'Ticket':",db["Ticket"].duplicated().sum()/db.shape[0]*100,"%")

Duplicados en 'Ticket': 23.56902356902357 %


In [49]:
# Mostrar filas con la columna de ticket igual
db[db["Ticket"]=="CA. 2343"]


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
Name,,,,,,,,,
"Sage, Master. Thomas Henry",0,3,male,NaN,8,2,CA. 2343,69.55,S
"Sage, Miss. Constance Gladys",0,3,female,NaN,8,2,CA. 2343,69.55,S
"Sage, Mr. Frederick",0,3,male,NaN,8,2,CA. 2343,69.55,S
"Sage, Mr. George John Jr",0,3,male,NaN,8,2,CA. 2343,69.55,S
"Sage, Miss. Stella Anna",0,3,female,NaN,8,2,CA. 2343,69.55,S
"Sage, Mr. Douglas Bullen",0,3,male,NaN,8,2,CA. 2343,69.55,S
"Sage, Miss. Dorothy Edith ""Dolly""",0,3,female,NaN,8,2,CA. 2343,69.55,S


In [50]:
db[db["Ticket"]=="347082"]

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
Name,,,,,,,,,
"Andersson, Mr. Anders Johan",0,3,male,39.0,1,5,347082,31.275,S
"Andersson, Miss. Ellis Anna Maria",0,3,female,2.0,4,2,347082,31.275,S
"Andersson, Miss. Ingeborg Constanzia",0,3,female,9.0,4,2,347082,31.275,S
"Andersson, Miss. Sigrid Elisabeth",0,3,female,11.0,4,2,347082,31.275,S
"Andersson, Mrs. Anders Johan (Alfrida Konstantia Brogren)",0,3,female,39.0,1,5,347082,31.275,S
"Andersson, Miss. Ebba Iris Alfrida",0,3,female,6.0,4,2,347082,31.275,S
"Andersson, Master. Sigvard Harald Elias",0,3,male,4.0,4,2,347082,31.275,S


In [51]:
# Eliminar columna Ticket
db.drop("Ticket",axis=1,inplace=True)

In [52]:
# mapeamos las variables sex y embarked
db["Embarked"] = db["Embarked"].map({"S": 0, "C": 1, "Q": 2})
db["Sex"] = db["Sex"].map({"male": 0, "female": 1})

db.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Name,,,,,,,,
"Braund, Mr. Owen Harris",0,3,0,22.0,1,0,7.2500,0
"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",1,1,1,38.0,1,0,71.2833,1
"Heikkinen, Miss. Laina",1,3,1,26.0,0,0,7.9250,0
"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,1,35.0,1,0,53.1000,0
"Allen, Mr. William Henry",0,3,0,35.0,0,0,8.0500,0


In [53]:
# Crear una instancia del imputador MICE
mice_imputer = IterativeImputer(initial_strategy="mean", estimator=LinearRegression())

# Imputar los valores faltantes
db = pd.DataFrame(mice_imputer.fit_transform(db), columns=db.columns,index=db.index)

# Limitar los valores imputados al rango de 0 a 100
db["Age"] = np.clip(db["Age"], 0, 100)

# Convertir los valores imputados a enteros
db["Age"] = db["Age"].astype(int)

In [54]:
# Mirar % de datos nulos por columna
for col in db.columns:
    print(col,":",db[col].isnull().sum()/db.shape[0]*100,"%")

Survived : 0.0 %
Pclass : 0.0 %
Sex : 0.0 %
Age : 0.0 %
SibSp : 0.0 %
Parch : 0.0 %
Fare : 0.0 %
Embarked : 0.0 %
